# Artificial Neural Network

## Setup and Context

### Introduction

In this notebook is an Artificial Neural Network which identifies bank customers that are withdrawing their accounts from the bank. The dataset used is from [Kaggle](https://www.kaggle.com/datasets/santoshd3/bank-customers).

### Import Statements

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
tf.__version__

'2.15.0'

## Data Preprocessing

### Load the Data

In [3]:
df = pd.read_csv("./data/Churn Modeling.csv")

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [6]:
df.duplicated().sum()

0

### Getting Features and Targets

We are going to split our data into features and targets.
- **Features**, also known as predictors or independent variables, are the input variables that are used to make predictions or decisions. They represent different aspects or attributes of the data that are relevant to the problem being solved
- **Targets**, also known as labels or dependent variables, are the output variables that the model is trying to predict or classify based on the input features.

For the purpose of our Artifical Neural Network, the "RowNumber", "CustomerId" and "Surname" columns are not needed as they will not give us any valuable information. So we will not be including them in the features. Our target is the last column, "Exited".

In [7]:
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

In [8]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [9]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

### Encoding Categorical Data

Categorical data needs to be encoded into numerical form for machine learning algorithms because these algorithms typically require numerical input. Encoding ensures consistency, improves feature representation, and facilitates preprocessing. Common methods include ordinal encoding, one-hot encoding, label encoding and embedding, each suited to different types of categorical data and machine learning tasks.

We will be Label Encoding the "Gender" column. Label encoding converts categorical data into numerical form by assigning a unique integer label to each category.

In [10]:
le = LabelEncoder()

X[:, 2] = le.fit_transform(X[:, 2])

Now we will be One-Hot Encoding the "Geography" column. One-hot encoding is a method used to represent categorical data as binary vectors. 

In [11]:
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")

X = np.array(ct.fit_transform(X))

In [12]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

### Splitting into Training and Validation Sets

We split the data into training and test sets. We are going to use a 80:20 split.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

### Feature Scaling

There are two main ways of feature scaling: **Standardization and Normalization**. We will be using Standardization here.

In [14]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Building the ANN

### Initializing the ANN

The Artificial Neural Network is a sequence of layers. Therefore we are going to intialize our model with the `Sequential` class.

In [15]:
ann = tf.keras.models.Sequential()

### Adding the Input Layer and the First Hidden Layer

The input layer would have one neuron for each column in our features. In our first hidden layer we want it to have 16 neurons. The activation function we will be using for this layer is the Rectified Linear Unit (ReLU).

In [16]:
ann.add(tf.keras.layers.Dense(units=16, activation="relu"))

### Adding the Second Hidden Layer

We are adding another layer to our model. Another Dense layer with 16 neurons and ReLU as the activation function.

In [17]:
ann.add(tf.keras.layers.Dense(units=16, activation="relu"))

### Adding the Output Layer

In the output layer we want just a single neuron with the Sigmoid activation function because we are solving a binary classification problem.

In [18]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

## Training the ANN

### Compiling the ANN

We are using an **Atom Optimizer** to perform **Stochastic Gradient Descent**. Our **Loss Function** is the **Binary Cross Entropy Loss**. We also using **Accuracy Metrics** because it is a classification problem.

In [19]:
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

### Fitting the ANN on the Training Set

Now we can train the model on our training set.

In [20]:
ann.fit(X_train, y_train, batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 878us/step - loss: 0.5068 - accuracy: 0.7634
Epoch 2/100
250/250 [==============================] - 0s 740us/step - loss: 0.4227 - accuracy: 0.8133
Epoch 3/100
250/250 [==============================] - 0s 805us/step - loss: 0.3975 - accuracy: 0.8324
Epoch 4/100
250/250 [==============================] - 0s 706us/step - loss: 0.3752 - accuracy: 0.8462
Epoch 5/100
250/250 [==============================] - 0s 819us/step - loss: 0.3584 - accuracy: 0.8550
Epoch 6/100
250/250 [==============================] - 0s 711us/step - loss: 0.3489 - accuracy: 0.8566
Epoch 7/100
250/250 [==============================] - 0s 808us/step - loss: 0.3429 - accuracy: 0.8624
Epoch 8/100
250/250 [==============================] - 0s 791us/step - loss: 0.3399 - accuracy: 0.8606
Epoch 9/100
250/250 [==============================] - 0s 743us/step - loss: 0.3371 - accuracy: 0.8619
Epoch 10/100
250/250 [==============================] - 0s 717us/step - l

## Making Predictions and Evaluating the Model

### Predicting the Test Results

Let us predict the test set and compare it to the actual values. The prediction would give us a value between 0 and 1 where 0 means that the model is certain that a customer will NOT leave the bank and 1 means that the mode is certain that a customer will leave the bank.

In [21]:
y_pred = ann.predict(X_test)

y_pred = y_pred > 0.5

63/63 [==============================] - 0s 866us/step


In [22]:
np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1)

array([[0, 1],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int64)

### Making the Confusion Matrix

The confusion matrix is a table that is often used to evaluate the performance of a classification model. It summarizes the predictions made by a classifier on a set of data for various classes against their actual labels.

- True Positive (TP): Instances correctly predicted as belonging to the positive class.
- False Positive (FP): Instances incorrectly predicted as belonging to the positive class (actually negative).
- True Negative (TN): Instances correctly predicted as belonging to the negative class.
- False Negative (FN): Instances incorrectly predicted as belonging to the negative class (actually positive).

In [23]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
cm

array([[1527,   60],
       [ 219,  194]], dtype=int64)

Let us check the accuracy of our model on the test data.

In [24]:
acc = accuracy_score(y_true=y_test, y_pred=y_pred)
acc

0.8605